In [38]:
import json, os, pandas as pd

In [39]:
profile_log = '/mydata/profiler_benchmark/pytorch_profiler_imagenet_subset_b512_gpu1/c4130-node.v100ubuntu20.prismgt-pg0.wisc.cloudlab.us_174279.1705996151567.pt.trace.json'
# Below event's duration si the wait time
marker_event_name = 'enumerate(DataLoader)#_MultiProcessingDataLoaderIter.__next__'
custom_log_dir = '/mydata/profiler_benchmark/custom_log_profiles_imagenet_subset_b512_gpu1'

In [40]:
# open as json
data = json.load(open(profile_log))

In [41]:
pytorch_profiler_wait_sum = 0
count = 0
for event in data['traceEvents']:
    if event['name'] == marker_event_name:
        count +=1
        pytorch_profiler_wait_sum += event['dur']
print(f'count: {count}')
print(f'pytorch_profiler_wait_sum: {pytorch_profiler_wait_sum} us')

count: 51
pytorch_profiler_wait_sum: 313031892 us


In [42]:
custom_log_wait_sum = 0
# walk through all files in the custom log dir
for root, dirs, files in os.walk(custom_log_dir):
    print(root)
    for file in files:
        print(file)
        # if file does not contain 'worker_pid' skip
        if 'worker_pid' in file:
            continue
        # read file as pandas csv
        df = pd.read_csv(os.path.join(root, file))
        # add headers as 'name', 'start', 'dur'
        df.columns = ['name', 'start', 'dur']

        # if name contains 'SBatchWait' then keep else remove
        df = df[df['name'].str.contains('SBatchWait')]

        # get duration sum
        custom_log_wait_sum = df['dur'].sum()/1000 # convert to us

        # print length of df
        print(f'len(df): {len(df)}')

print(f'custom_log_wait_sum: {custom_log_wait_sum} us')


/mydata/profiler_benchmark/custom_log_profiles_imagenet_subset_b512_gpu1
custom_log_worker_pid_1487029
custom_log_main_pid_1486988
len(df): 51
custom_log_wait_sum: 167712150.62 us


In [47]:
# get difference %
print(f'diff: {pytorch_profiler_wait_sum/custom_log_wait_sum:.2f}x')

diff: 1.87x


In [50]:
custom_log_wait_sum/1000/1000

167.71215062